In [2]:
import os
from dotenv import load_dotenv
from pathlib import Path

# Load .env from the project root
env_path = Path('..') / '.env'  # Go one directory up to locate .env
load_dotenv(dotenv_path=env_path)
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY']= "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"
# os.environ['LANGCHAIN_PROJECT']="pr-advanced-theism-85"

# Chunk Retrieval using Metadata filter

In [ ]:
import chromadb
from sentence_transformers import SentenceTransformer
import streamlit as st
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

from utils.chunk_doc import get_retriever, get_vector_store



# llm = ChatOpenAI(
#     model="gpt-4o-mini",
#     api_key=st.secrets["OpenAI_key"]
# )

vector_store = get_vector_store(path = "db/pdfs")

query = "What is Insertion Sort?"
metadata_filter = {"keywords": "Insertion"}
# response = vector_store.search(query,search_type="mmr", k=5, fetch_k=10)
# print(response)
found_docs = vector_store.max_marginal_relevance_search(query, filter=metadata_filter)
print(found_docs)
for i, doc in enumerate(found_docs):
    print(f"{i + 1}.", doc.page_content, "\n")

[Document(metadata={'source': 'data\\md\\dsa.md'}, page_content='_CHAPTER 8. SORTING_ 67\n\n###### 8.4 Insertion Sort\n\nInsertion sort is a somewhat interesting algorithm with an expensive runtime of\n_O(n[2]). It can be best thought of as a sorting scheme similar to that of sorting_\na hand of playing cards, i.e. you take one card and then look at the rest with\nthe intent of building up an ordered set of cards in your hand.\n\n4 75 74\n\n4 75 74 2 54 4 75 74 2 54 4 75 74 2 54\n\n2 54\n\n4 74 75 2 54 2 4 74 75 54 2 4 54 74 75\n\nFigure 8.4: Insertion Sort Iterations\n\n1) algorithm Insertionsort(list)\n2) **Pre:** _list_ =\n_̸_ _∅_\n3) **Post: list has been sorted into values of ascending order**\n4) _unsorted_ 1\n_←_\n5) **while unsorted < list.Count**\n6) _hold_ _list[unsorted]_\n_←_\n7) _i_ _unsorted_ 1\n_←_ _−_\n8) **while i** 0 and hold < list[i]\n_≥_\n9) _list[i + 1]_ _list[i]_\n_←_\n10) _i_ _i_ 1\n_←_ _−_\n11) **end while**\n12) _list[i + 1]_ _hold_\n_←_\n13) _unsorted_ _unsor

# Keyword Generation using LLAMA 3.2

In [2]:
from langchain_ollama import OllamaLLM
from langchain_chroma import Chroma
from langchain.embeddings.base import Embeddings
from sentence_transformers import SentenceTransformer
from typing import List
import os
from langchain_openai import ChatOpenAI

class MyEmbeddings(Embeddings):
        def __init__(self):
            self.model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    
        def embed_documents(self, texts: List[str]) -> List[List[float]]:
            return [self.model.encode(t).tolist() for t in texts]
        
        def embed_query(self, query: str) -> List[float]:
            return self.model.encode(query).tolist()

embedding_func = MyEmbeddings()

# Initialize Ollama LLM
llm = OllamaLLM(
    # model="gemma2:2b",
    model = "llama3.2:latest",
    base_url="http://localhost:11434"  # Adjust this URL if needed
)

from langchain_core.prompts import ChatPromptTemplate

# Define a prompt template for Ollama to generate keywords
# Gemma2:2b Template
# keyword_prompt_template = ChatPromptTemplate.from_messages(
#     [
#         ("system", "You are an assistant that generates keywords for a chunk of text. The keywords must be single words or two-word phrases. Format the output as: ['keyword1', 'keyword2']"),
#         ("human", "Extract relevant keywords for the following chunk:\n\n{chunk_text}")
#     ]
# )

# Llama3.2:1b Template
keyword_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an assistant that generates keywords for a chunk of text. "
                   "Your response should only contain keywords in json format."),
        ("human", "Extract relevant keywords from the following chunk:\n\n{chunk_text}")
    ]
)

chain = keyword_prompt_template | llm

# Initialize ChromaDB client
vector_store = Chroma(
    # client = client,
    collection_name="markdown_chunks_collection",
    embedding_function=embedding_func,
    persist_directory = "../db/test_pdfs",
    # other params...
)

def split_chunks():
    try:
        from pathlib import Path
        from langchain_core.documents import Document
        from langchain_text_splitters import RecursiveCharacterTextSplitter as Rec
        
        # Path to markdown directory
        md_dir = Path("../data/md/")
        chunk_id_counter = 1  # Initialize a counter for unique chunk IDs
        ids = []
        documents = []

        # Loop through all markdown files in the md directory
        for md_file in md_dir.glob("*.md"):
            with open(md_file, "r") as f:
                md_content = f.read()

            # Chunk the markdown content
            text_splitter = Rec(
                chunk_size=2000,
                chunk_overlap=500,
                length_function=len,
                add_start_index=True
            )
            chunks = text_splitter.split_text(md_content)
            
            for chunk in chunks:
                # Generate keywords for the chunk using Ollama
                response = chain.invoke({"chunk_text": chunk})
                
                import json
                # Convert the JSON string to a Python dictionary
                dictionary_output = json.loads(response)

                # Access the "keywords" list
                keywords_list = dictionary_output["keywords"]
                
                # Create a Document object with metadata for the chunk, including keywords
                document_to_add = Document(
                    page_content=chunk,
                    metadata={"source": str(md_file), "keywords": str(keywords_list)}
                )
                
                documents.append(document_to_add)
                ids.append(str(chunk_id_counter))  # Add document ID to the list
                chunk_id_counter += 1  # Increment the ID counter
        
        # Assuming vector_store is defined and initialized elsewhere
        vector_store.add_documents(documents=documents, ids=ids)
    except Exception as e:
        print(f"Error: {e}")
        
if __name__ == "__main__":
    # split_chunks()
    
    # print(response)

    response = chain.invoke({"chunk_text": """ ###### 2.1.1 Insertion

In general when people talk about insertion with respect to linked lists of any
form they implicitly refer to the adding of a node to the tail of the list. When
you use an API like that of DSA and you see a general purpose method that
adds a node to the list, you can assume that you are adding the node to the tail
of the list not the head.

Adding a node to a singly linked list has only two cases:

1. head = in which case the node we are adding is now both the head and
_∅_
_tail of the list; or_

2. we simply need to append our node onto the end of the list updating the
_tail reference appropriately._

1) algorithm Add(value)
2) **Pre: value is the value to add to the list**
3) **Post: value has been placed at the tail of the list**
4) _n_ node(value)
_←_
5) **if head =**
_∅_
6) _head_ _n_
_←_
7) _tail_ _n_
_←_
8) **else**
9) _tail.Next_ _n_
_←_
10) _tail_ _n_
_←_
11) **end if**
12) end Add

As an example of the previous algorithm consider adding the following sequence of integers to the list: 1, 45, 60, and 12, the resulting list is that of
Figure 2.2.

###### 2.1.2 Searching

Searching a linked list is straightforward: we simply traverse the list checking
the value we are looking for with the value of each node in the linked list. The
algorithm listed in this section is very similar to that used for traversal in 2.1.4.
_§_


-----

_CHAPTER 2. LINKED LISTS_ 11

1) algorithm Contains(head, value)
2) **Pre: head is the head node in the list**
3) _value is the value to search for_
4) **Post: the item is either in the linked list, true; otherwise false**
5) _n_ _head_
_←_
6) **while n** = **and n.Value** = value
_̸_ _∅_ _̸_
7) _n_ _n.Next_
_←_
8) **end while**
9) **if n =**
_∅_
10) **return false**
11) **end if**
12) **return true**
13) end Contains

###### 2.1.3 Deletion

Deleting a node from a linked list is straightforward but there are a few cases
we need to account for:

1. the list is empty; or

2. the node to remove is the only node in the linked list; or

3. we are removing the head node; or

4. we are removing the tail node; or

5. the node to remove is somewhere in between the head and tail; or

6. the item to remove doesn’t exist in the linked list

The algorithm whose cases we have described will remove a node from anywhere within a list irrespective of whether the node is the head etc. If you know
that items will only ever be removed from the head or tail of the list then you
can create much more concise algorithms. In the case of always removing from
the front of the linked list deletion becomes an O(1) operation."""})

    # print(dictionary_output)
    print(response)

{"keywords": ["insertion", "linked lists", "add node", "tail", "head", "algorithm", "Add", "Contains", "searching", "deletion", "Remove"]}


# Multi Query

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_chroma import Chroma
from langchain.embeddings.base import Embeddings
from typing import List
import os
import streamlit as st
from langchain_openai import ChatOpenAI
from langchain_ollama import OllamaLLM
from sentence_transformers import SentenceTransformer

class MyEmbeddings(Embeddings):
        def __init__(self):
            self.model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    
        def embed_documents(self, texts: List[str]) -> List[List[float]]:
            return [self.model.encode(t).tolist() for t in texts]
        
        def embed_query(self, query: str) -> List[float]:
            return self.model.encode(query).tolist()

embedding_func = MyEmbeddings()


# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by a single newline. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser

gpt4 = ChatOpenAI(
    model="gpt-4o-mini",
    # api_key=os.environ.get("OPENAI_API_KEY"),
    api_key=st.secrets["OpenAI_key"],
    temperature=0
)

# llm = OllamaLLM(model="gemma2:2b", base_url="http://localhost:11434")
# llm = OllamaLLM(model="gemma2:2b", base_url="http://localhost:11434")

generate_queries = (
    prompt_perspectives 
    | gpt4
    | StrOutputParser() 
    | (lambda x: [line for line in x.split("\n") if line.strip() != ""])  # Ensure empty strings are removed
)

from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Initialize ChromaDB client
vector_store = Chroma(
    # client = client,
    collection_name="markdown_chunks_collection",
    embedding_function=embedding_func,
    persist_directory = "../db/pdfs",
    # other params...
)

retriever = vector_store.as_retriever()

# Retrieve
question = "What is Insertion Sort?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
# print(docs)
len(docs)

from operator import itemgetter

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | prompt
    | gpt4
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})


'Insertion Sort is a sorting algorithm that builds a sorted array (or list) one item at a time. It is similar to the way one might sort a hand of playing cards. The algorithm works by taking one element from the unsorted portion of the array and inserting it into the correct position in the sorted portion. \n\nThe process can be summarized as follows:\n1. Start with the first element, which is considered sorted.\n2. Take the next element from the unsorted portion and compare it with the elements in the sorted portion.\n3. Shift the sorted elements as necessary to make room for the new element.\n4. Insert the new element into its correct position.\n5. Repeat this process until all elements are sorted.\n\nThe time complexity of Insertion Sort is O(n²) in the average and worst cases, making it less efficient for large datasets compared to more advanced algorithms. However, it is efficient for small datasets or partially sorted arrays and is a stable sorting algorithm, meaning that it pres

# RAG-Fusion

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_chroma import Chroma
from langchain.embeddings.base import Embeddings
from typing import List
import os
from langchain_openai import ChatOpenAI
from sentence_transformers import SentenceTransformer

class MyEmbeddings(Embeddings):
        def __init__(self):
            self.model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    
        def embed_documents(self, texts: List[str]) -> List[List[float]]:
            return [self.model.encode(t).tolist() for t in texts]
        
        def embed_query(self, query: str) -> List[float]:
            return self.model.encode(query).tolist()

embedding_func = MyEmbeddings()


# RAG-Fusion: Related
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=os.environ.get("OPENAI_API_KEY"),
    # api_key=st.secrets["OpenAI_key"],
    temperature=0
)

generate_queries = (
    prompt_rag_fusion 
    | llm
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents 
        and an optional parameter k used in the RRF formula """
    
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

# Initialize ChromaDB client
vector_store = Chroma(
    # client = client,
    collection_name="markdown_chunks_collection",
    embedding_function=embedding_func,
    persist_directory = "../db/pdfs",
    # other params...
)

retriever = vector_store.as_retriever()

# Retrieve
question = "What is Insertion Sort?"
retrieval_chain_rag_fusion  = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain.invoke({"question":question})
# print(docs)
len(docs)

from operator import itemgetter

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})


'Insertion Sort is a sorting algorithm that builds a sorted array (or list) one element at a time. It starts by treating the first entry as an already sorted array and then compares the next entry with the sorted portion. If the next entry is smaller than the sorted entries, it is inserted into the correct position by shifting the larger entries up to make space. This process continues for each subsequent entry until the entire array is sorted.\n\nThe general algorithm for Insertion Sort can be described as follows:\n\n1. Start with the first element as sorted.\n2. For each subsequent element, compare it with the elements in the sorted portion.\n3. Shift the larger elements up to make space for the new element.\n4. Insert the new element into its correct position.\n5. Repeat until all elements are sorted.\n\nInsertion Sort is considered stable, meaning that it preserves the relative order of items with equal keys. Its average and worst-case time complexity is O(n²), making it less effi

In [3]:
from utils.bm25_ranking import find_closest_chunks_bm25, new_bm25, re_rank_chunks_with_embeddings
print("OLD BM25")
results_top_n = find_closest_chunks_bm25(query, results2, top_n=10)
for res in results_top_n:
    print(res['score']," ",res['id'],"\n")
    # print(res['document'],"\n")


print("NEW BM25")
bm25_results = new_bm25(query, results2, top_n=10)
sorted_results = re_rank_chunks_with_embeddings(query, bm25_results)

# print(results_top_n)
# for res in results_top_n:
#     print(res['score']," ",res['id'],"\n")
#     print(res['document'],"\n")

# organised_list = {}
# for item in results2:
#     print(item)
#     # organised_list[results2[item]] = ## id and document
# print(organised_list)
# print(results_top_n)
# print(md_collection.get(ids="297"))

OLD BM25
14.584834019480919   377 

12.523017371922746   378 

12.050891594269249   375 

11.880403985989389   866 

11.260478820127473   535 

11.123203021744485   815 

11.035082004738484   814 

11.002614755875888   406 

10.965935890038589   260 

10.352130470797842   805 

NEW BM25
Chunk ID: 815, BM25 Score: 11.123203021744485, Embedding Score: 0.4674035310745239, Source: data\md\dsa.md
Most libraries provide implementations of unordered sets and so DSA does
not; we simply mention it here to disambiguate between an unordered set and
ordered set.

We will only look at insertion for an unordered set and cover briefly why a
hash table is an efficient data structure to use for its implementation.

###### 5.1.1 Insertion

An unordered set can be efficiently implemented using a hash table as its backing
data structure. As mentioned previously we only add an item to a set if that
item is not already in the set, so the backing data structure we use must have
a quick look up and insertion 

In [22]:
print(md_collection.get(ids=['220']))

{'ids': ['220'], 'embeddings': None, 'metadatas': [{'source': 'data\\md\\cp1.md'}], 'documents': ['best possible acorns collected when Jayjay is at this height. The bottom-up DP code that requires\n\nonly 2000 = 2K states and time complexity of 2000 2000 = 4M is as follow:\n_×_\n\nfor (int tree = 0; tree < t; tree++) // initialization\ndp[h] = max(dp[h], acorn[tree][h]);\nfor (int height = h - 1; height >= 0; height--)\nfor (int tree = 0; tree < t; tree++) {\nacorn[tree][height] +=\nmax(acorn[tree][height + 1], // from this tree, +1 above\n((height + f <= h) ? dp[height + f] : 0)); // best from tree at height + f\ndp[height] = max(dp[height], acorn[tree][height]); // update this too\n}\nprintf("%d\\n", dp[0]); // solution will be here\n\nLesson: When na¨ıve DP states are too large causing the overall DP time complexity not-doable,\n\nthink of different ways other than the obvious to represent the possible states. Remember that no\n\nprogramming contest problem is unsolvable, the proble